In [2]:
%matplotlib inline

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import os

# Huffman Coding
## Author: Tsvetan Dimitrov


### Abstract
TODO

###  Lossy vs Lossless Compression
In this section we will look at lossy vs lossless compression and the advantages and disadvantages of both methods. There is no right or wrong method. It all comes down to taking some notice of a number of different factors. 

#### Lossy Compression
The first type is lossy compression which refers to some of the data from the original file being lost after compression is executed. The process is irreversible and once you convert to lossy, you cannot go back. And the more you compress it, the more degradation occurs. JPEG and GIF are both lossy image formats. One of the biggest obvious benefits to using lossy compression is that it results in a significantly reduced file size (smaller than lossless compression method), but it also means there is quality loss. Most tools, plugins, and software out there will let you choose the degree of compression you want to use.

#### Lossless Compression
The other type is lossless compression which refers to compression without any data or quality loss. All of original data can be recovered when the file is uncompressed. RAW, BMP, GIF, and PNG are all lossless image formats. The big benefit to lossless compression is that you can retain the original quality of your data or images and still achieve a smaller file size. This is generally the technique of choice for text or spreadsheet files, where losing words or financial data could pose a problem. In this article we will explore and implement a lossless technique called Huffman Coding. 

### Information Entropy
In information theory, the major goal is for one person (a transmitter) to convey some message (over a channel) to another person (the receiver). To do so, the transmitter sends a series (possibly just one) partial messages that give clues towards the original message. The information content of one of these partial messages is a measure of how much uncertainty this resolves for the receiver. Let us try a simple experiment. We will assume that the weather is equally probable to be at any one of 4 possible states at any given moment. This translates to having the same probability of a state occurring which is 1/4 in our case.

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/4  |  1/4   |  1/4  |  1/4  |

Now the question is what is the minimal number of bits we can use to store each of these states based on their probability? Our probability can be expressed as follows: 
$$\frac{1}{4} = \frac{1}{2^2} = 2^{-2}$$

$$\text{minimum number of bits} = -\log (p) = -\log \frac{1}{4} = 2$$
So we can now add codes to our table for each weather state:

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/4  |  1/4   |  1/4  |  1/4  |
| code        |  00   |  01    |   10  |   11  |

Each code has to uniquely identify the data that it is encoding, otherwise we will not be able to decode it correctly. Let us now try to change the probabilities and calculate their corresponding codes:

|             | Sunny | Cloudy | Rainy | Foggy |
|-------------|-------|--------|-------|-------|
| probability |  1/2  |  1/4   |  1/8  |  1/8  |
| code        |  0    |  10    |  110  |  111  |


The fact that we can derive from our experiment is that the lower the probability value of a data source, the more information a message transfer event has to carry than a data source with a higher probability value. A partial message that cuts the number of possibilities in half transmits one bit of information about the message. In essence, the "information content" can be viewed as how much useful information the message actually contains. The entropy, in this context, is the expected number of bits of information contained in each message, taken over all possibilities for the transmitted message. In information theory entropy is denoted with $H$ and has the following definition:
$$ H(X) = -p\log p$$
This gives us the weighted average of bits for the current partial message or state.

### Algorithm


In [ ]:
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __cmp__(self, )

### Conclusion

### Future Work

### References